In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from benchmarkrewriter.benchmark_parser import BenchmarkParser, WorkerBenchmarkParser, WorkerEncoding
import os

path_to_results = r'C:\Users\huda\Downloads\\'
path_to_results = r'C:\Users\huda\Downloads\ga\ga\\'
benchmark_path = r'C:\Users\huda\Documents\GitHub\scheduling_model_jrc\code\upgrades\benchmarks_with_workers'

In [2]:
import math
def multiset_permutations(sequence : list) -> int:
    a = math.factorial(len(sequence))
    counts = [sequence.count(i) for i in range(min(sequence), max(sequence)+1)]
    b = math.prod([math.factorial(counts[x]) for x in range(len(counts))])
    return a/b

print(multiset_permutations([0, 0, 1, 1]))
print(multiset_permutations([1, 1, 2, 2]))
print(multiset_permutations([1, 1]))
print(multiset_permutations([1, 2]))
print(multiset_permutations([1]))
print(multiset_permutations([1, 1, 2]))
print(multiset_permutations([1, 2, 1]))
print(multiset_permutations([1, 2, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7 ,8, 8, 8]))

6.0
6.0
1.0
2.0
1.0
3.0
3.0
1.30729041081037e+29


In [3]:
def worker_flexibility(benchmark : WorkerEncoding):
    n_assignments = 0
    m = benchmark.n_machines()
    o = benchmark.n_operations()
    durations = benchmark.durations()
    w = durations.shape[2]
    for i in range(len(durations)):
        for j in range(len(durations[i])):
            for k in range(len(durations[i][j])):
                if durations[i][j][k] > 0:
                    n_assignments += 1
    average_assignments = n_assignments / o
    return average_assignments / (m*w)


def get_flexibility_and_dv_worker(benchmark):
    all = 0
    unique = []
    machines_available = 0
    durations = benchmark.durations()
    for i in range(len(durations)):
        for j in range(len(durations[i])):
            for k in range(len(durations[i][j])):
                if durations[i][j][k] > 0:
                    if durations[i][j][k] not in unique:
                        unique.append(durations[i][j][k])
                    all += 1
            if any([x > 0 for x in durations[i][j]]):
                machines_available+=1
    return worker_flexibility(benchmark), len(unique) / all

def remap(name):
    if name.startswith('_'):
        name = name[1:]
    values = name.split('_')
    if values[0].startswith('Behnke'):
        return 'Behnke'+values[1]
    if values[0].startswith('Brandimarte'):
        return 'BrandimarteMk'+values[1]
    if values[0].startswith('Chambers'):
        return 'ChambersBarnes'+values[1]
    if values[0].startswith('HurinkS'):
        return 'HurinkSdata'+values[1]
    if values[0].startswith('HurinkE'):
        return 'HurinkEdata'+values[1]
    if values[0].startswith('HurinkR'):
        return 'HurinkRdata'+values[1]
    if values[0].startswith('HurinkV'):
        return 'HurinkVdata'+values[1]
    if values[0].startswith('DP'):
        return 'DPpaulli'+values[1]
    if values[0].startswith('Kacem'):
        return 'Kacem'+values[1]
    if values[0].startswith('Fattahi'):
        return 'Fattahi'+values[1]
    return name

def read_benchmarks_workers(path):
    result = dict()
    benchmarks = os.listdir(path)
    for benchmark in benchmarks:
        #parser = BenchmarkParser()
        parser = WorkerBenchmarkParser()
        data = parser.parse_benchmark(path + '/' + benchmark)
        #f, dv = get_flexibility_and_dv(data)
        f, dv = get_flexibility_and_dv_worker(data)
        instance_name = remap(benchmark[2:-12])
        result[instance_name] = {'n_operations': data.n_operations(), 'flexibility': f, 'duration_variety': dv, 'n_machines': data.n_machines()}

    return result

In [4]:
benchmark_data = read_benchmarks_workers(benchmark_path)

In [5]:
import os
def rewrite(category):
    files = os.listdir(path_to_results+category)
    json_data = {}
    for file in files:
        with open(path_to_results + category + "\\" + file, 'r') as f:
            content = f.read()
            content = '{"' + content[:7] + '":' + content[8:-1] + ']' + '}'
            json_data[file] = content
    return json_data

In [6]:
def remap(name):
    if name.startswith('_'):
        name = name[1:]
    values = name.split('_')
    if values[1].startswith('Behnke'):
        return 'Behnke'+values[-1]
    if values[1].startswith('Brandimarte'):
        return 'BrandimarteMk'+values[-1]
    if values[1].startswith('Chambers'):
        return 'ChambersBarnes'+values[-1]
    if values[1].startswith('Hurink') and values[2].startswith('s'):
        return 'Hurink_sdata_'+values[-1]
    if values[1].startswith('Hurink') and values[2].startswith('e'):
        return 'Hurink_edata_'+values[-1]
    if values[1].startswith('Hurink') and values[2].startswith('r'):
        return 'Hurink_rdata_'+values[-1]
    if values[1].startswith('Hurink') and values[2].startswith('v'):
        return 'Hurink_vdata_'+values[-1]
    if values[1].startswith('DP'):
        return 'DPpaulli'+values[-1]
    if values[1].startswith('Kacem'):
        return 'Kacem'+values[-1]
    if values[1].startswith('Fattahi'):
        return 'Fattahi'+values[-1]
    return name

In [7]:
import json
results_categories = os.listdir(path_to_results)
results_categories = ['local', 'nolocal']
results = dict()
for category in results_categories:
    results[category] = dict()
    instances = os.listdir(path_to_results + category)
    for instance in instances:
        #with open(path_to_results + category + '\\' + instance, 'r') as f:
        #TODO: turn into valid json
        j_data = rewrite(category)
        for instance in j_data:
            ga_data = json.loads(j_data[instance])
            # TODO: gather best, average, ...
            if instance not in results[category]:
                results[category][remap(instance[:-17])] = ga_data


In [8]:
for result in results:
    print(results[result].keys())

dict_keys(['Behnke42', 'Behnke46', 'Behnke60', 'BrandimarteMk12', 'BrandimarteMk14', 'BrandimarteMk7', 'Hurink_sdata_18', 'Hurink_sdata_1', 'Hurink_sdata_38', 'Hurink_sdata_40', 'Hurink_sdata_54', 'Hurink_sdata_61', 'Hurink_sdata_63', 'Hurink_edata_1', 'Hurink_edata_6', 'Hurink_rdata_28', 'Hurink_rdata_38', 'Hurink_rdata_50', 'Hurink_vdata_18', 'Hurink_vdata_5', 'DPpaulli15', 'DPpaulli18', 'DPpaulli1', 'DPpaulli9', 'ChambersBarnes10', 'Kacem3', 'Kacem4', 'Fattahi14', 'Fattahi20'])
dict_keys(['Behnke42', 'Behnke46', 'Behnke60', 'BrandimarteMk12', 'BrandimarteMk14', 'BrandimarteMk7', 'Hurink_sdata_18', 'Hurink_sdata_1', 'Hurink_sdata_38', 'Hurink_sdata_40', 'Hurink_sdata_54', 'Hurink_sdata_61', 'Hurink_sdata_63', 'Hurink_edata_1', 'Hurink_edata_6', 'Hurink_rdata_28', 'Hurink_rdata_38', 'Hurink_rdata_50', 'Hurink_vdata_18', 'Hurink_vdata_5', 'DPpaulli15', 'DPpaulli18', 'DPpaulli1', 'DPpaulli9', 'ChambersBarnes10', 'Kacem3', 'Kacem4', 'Fattahi14', 'Fattahi20'])


In [9]:
best_overall = dict()
average_overall = dict()
tied_overall = []
average_tied_overall = []
used = []
for instance in benchmark_data:
    if instance in results['local'] or instance in results['nolocal']:
        best_overall[instance] = ('None', float('inf'))
        average_overall[instance] = ('None', float('inf'))
    for category in results:
        
        if instance in results[category]:
            if instance not in used: 
                used.append(instance)
            # TODO: decide what to do with instances with multiple best, also do same for average
            best = float('inf')
            average = 0
            for result in results[category][instance]['results']:
                if result['Result']['Fitness']['Makespan'] < best:
                    best = result['Result']['Fitness']['Makespan']
                average += result['Result']['Fitness']['Makespan']
            average /= len(results[category][instance]['results'])
            if best < best_overall[instance][1]:
                best_overall[instance] = (category, best)
            elif best == best_overall[instance][1]:
                tied_overall.append(instance)
                best_overall[instance] = ('None', float('inf'))
            if average < average_overall[instance][1]:
                average_overall[instance] = (category, average)
            elif average == average_overall[instance][1]:
                average_tied_overall.append(instance)
                average_overall[instance] = ('None', float('inf'))

In [10]:
len(used)

29

In [11]:
unused = []
for instance in list(results['local'].keys()):
    if instance not in used:
        unused.append(instance)

In [12]:
local_win = [best_overall[x] for x in best_overall if best_overall[x][0] == 'local']
nolocal_win = [best_overall[x] for x in best_overall if best_overall[x][0] == 'nolocal']
local_average_win = [average_overall[x] for x in average_overall if average_overall[x][0] == 'local']
nolocal_average_win = [average_overall[x] for x in average_overall if average_overall[x][0] == 'nolocal']

In [13]:
len(best_overall.keys())

29

In [14]:
for result in best_overall:
    if result[0] == 'nolocal':
        print('found')

In [15]:
print(f'Local better: {len(local_win)}')
print(f'No Local better: {len(nolocal_win)}')
print(f'Average Local better: {len(local_average_win)}')
print(f'Average No Local better: {len(nolocal_average_win)}')

Local better: 7
No Local better: 18
Average Local better: 1
Average No Local better: 25


In [16]:
local_average_win_instances = [x for x in average_overall if average_overall[x][0] == 'local']

In [17]:
nolocal_average_win_instances = [x for x in average_overall if average_overall[x][0] == 'nolocal']

In [18]:
import statistics
compare = [local_average_win_instances, nolocal_average_win_instances, average_tied_overall]
labels = ['local_win', 'no_local_win', 'tied']
for data, label in zip(compare, labels):
    ops = []
    f = []
    dv = []
    m = []
    for instance in data:
        ops.append(benchmark_data[instance]['n_operations'])
        f.append(benchmark_data[instance]['flexibility'])
        dv.append(benchmark_data[instance]['duration_variety'])
        m.append(benchmark_data[instance]['n_machines'])
    print(f'{label.upper()} - {len(ops)} - {(len(ops)/29)*100 :.2f}%')
    if len(ops) <= 1:            
        print(f'Operations: Mean: {ops[0]}, Median: {ops[0]}, Stdev: {0}')
        print(f'Machines: Mean: {m[0]}, Median: {m[0]}, Stdev: {0}')
        print(f'Flexibility: Mean: {f[0]}, Median: {f[0]}, Stdev: {0}')
        print(f'Duration Variety: Mean: {dv[0]}, Median: {dv[0]}, Stdev: {0}')
    else:
        print(f'Operations: Mean: {statistics.mean(ops)}, Median: {statistics.median(ops)}, Stdev: {statistics.stdev(ops)}, Min: {min(ops)}, Max: {max(ops)}')
        print(f'Machines: Mean: {statistics.mean(m)}, Median: {statistics.median(m)}, Stdev: {statistics.stdev(m)}, Min: {min(m)}, Max: {max(m)}')
        print(f'Flexibility: Mean: {statistics.mean(f)}, Median: {statistics.median(f)}, Stdev: {statistics.stdev(f)}, Min: {min(f)}, Max: {max(f)}')
        print(f'Duration Variety: Mean: {statistics.mean(dv)}, Median: {statistics.median(dv)}, Stdev: {statistics.stdev(dv)}, Min: {min(dv)}, Max: {max(dv)}')
    print()

LOCAL_WIN - 1 - 3.45%
Operations: Mean: 277, Median: 277, Stdev: 0
Machines: Mean: 15, Median: 15, Stdev: 0
Flexibility: Mean: 0.05245596761842249, Median: 0.05245596761842249, Stdev: 0
Duration Variety: Mean: 0.005005213764337852, Median: 0.005005213764337852, Stdev: 0

NO_LOCAL_WIN - 25 - 86.21%
Operations: Mean: 163.16, Median: 100, Stdev: 129.45677528297492, Min: 21, Max: 500
Machines: Mean: 14.56, Median: 10, Stdev: 17.342337404936703, Min: 4, Max: 60
Flexibility: Mean: 0.17408299731280397, Median: 0.1361, Stdev: 0.12126970140432095, Min: 0.03528619528619528, Max: 0.5384523809523809
Duration Variety: Mean: 0.14674748589543474, Median: 0.04344193817878028, Stdev: 0.2150092971036988, Min: 6.217652662025811e-05, Max: 0.7348066298342542

TIED - 3 - 10.34%
Operations: Mean: 34, Median: 36, Stdev: 3.4641016151377544, Min: 30, Max: 36
Machines: Mean: 7.333333333333333, Median: 6, Stdev: 2.309401076758503, Min: 6, Max: 10
Flexibility: Mean: 0.2470891632373114, Median: 0.11728395061728394,

In [19]:
local_win_instances = [x for x in best_overall if best_overall[x][0] == 'local']
nolocal_win_instances = [x for x in best_overall if best_overall[x][0] == 'nolocal']
compare = [local_win_instances, nolocal_win_instances, tied_overall]
labels = ['local_win', 'no_local_win', 'tied']
for data, label in zip(compare, labels):
    ops = []
    f = []
    dv = []
    m = []
    for instance in data:
        ops.append(benchmark_data[instance]['n_operations'])
        f.append(benchmark_data[instance]['flexibility'])
        dv.append(benchmark_data[instance]['duration_variety'])
        m.append(benchmark_data[instance]['n_machines'])
    print(f'{label.upper()} - {len(ops)} - {(len(ops)/29)*100 :.2f}%')
    if len(ops) <= 1:            
        print(f'Operations: Mean: {ops[0]}, Median: {ops[0]}, Stdev: {0}')
        print(f'Machines: Mean: {m[0]}, Median: {m[0]}, Stdev: {0}')
        print(f'Flexibility: Mean: {f[0]}, Median: {f[0]}, Stdev: {0}')
        print(f'Duration Variety: Mean: {dv[0]}, Median: {dv[0]}, Stdev: {0}')
    else:
        print(f'Operations: Mean: {statistics.mean(ops)}, Median: {statistics.median(ops)}, Stdev: {statistics.stdev(ops)}, Min: {min(ops)}, Max: {max(ops)}')
        print(f'Machines: Mean: {statistics.mean(m)}, Median: {statistics.median(m)}, Stdev: {statistics.stdev(m)}, Min: {min(m)}, Max: {max(m)}')
        print(f'Flexibility: Mean: {statistics.mean(f)}, Median: {statistics.median(f)}, Stdev: {statistics.stdev(f)}, Min: {min(f)}, Max: {max(f)}')
        print(f'Duration Variety: Mean: {statistics.mean(dv)}, Median: {statistics.median(dv)}, Stdev: {statistics.stdev(dv)}, Min: {min(dv)}, Max: {max(dv)}')
    print()

LOCAL_WIN - 7 - 24.14%
Operations: Mean: 188.85714285714286, Median: 196, Stdev: 107.59558319842652, Min: 50, Max: 300
Machines: Mean: 9, Median: 10, Stdev: 3.4641016151377544, Min: 5, Max: 15
Flexibility: Mean: 0.2209620386425948, Median: 0.13163265306122449, Stdev: 0.17152046168410268, Min: 0.05245596761842249, Max: 0.5384523809523809
Duration Variety: Mean: 0.05249784127076675, Median: 0.02128530495292673, Stdev: 0.06026222063565847, Min: 0.005005213764337852, Max: 0.16351118760757316

NO_LOCAL_WIN - 18 - 62.07%
Operations: Mean: 163.05555555555554, Median: 100.0, Stdev: 140.79123979358369, Min: 21, Max: 500
Machines: Mean: 17, Median: 10.0, Stdev: 20.00294096023941, Min: 4, Max: 60
Flexibility: Mean: 0.15576236421479356, Median: 0.13719285714285714, Stdev: 0.0956102440874998, Min: 0.03528619528619528, Max: 0.31342857142857145
Duration Variety: Mean: 0.17987301786499485, Median: 0.046745030687080226, Stdev: 0.24491793142706522, Min: 6.217652662025811e-05, Max: 0.7348066298342542

TI

In [20]:
print(max([benchmark_data[instance]['n_machines'] for instance in used]))
print(max([benchmark_data[instance]['n_operations'] for instance in used]))
print(max([benchmark_data[instance]['flexibility'] for instance in used]))
print(max([benchmark_data[instance]['duration_variety'] for instance in used]))

60
500
0.5384523809523809
0.7348066298342542


In [21]:
print(min([benchmark_data[instance]['n_machines'] for instance in used]))
print(min([benchmark_data[instance]['n_operations'] for instance in used]))
print(min([benchmark_data[instance]['flexibility'] for instance in used]))
print(min([benchmark_data[instance]['duration_variety'] for instance in used]))

4
21
0.03528619528619528
6.217652662025811e-05


Dissimilarity

In [5]:
import os
def rewrite(category):
    files = os.listdir(path_to_results+category)
    json_data = {}
    for file in files:
        with open(path_to_results + category + "\\" + file, 'r') as f:
            content = f.read()
            content = '{"' + content[1:7] + '"' + content[8:-1] + ']' + '}'
            json_data[file] = content
    return json_data

In [20]:
def remap(name):
    if name.startswith('_'):
        name = name[1:]
    values = name.split('_')
    if values[1].startswith('Behnke'):
        return 'Behnke'+values[-1]
    if values[1].startswith('Brandimarte'):
        return 'BrandimarteMk'+values[-1]
    if values[1].startswith('Chambers'):
        return 'ChambersBarnes'+values[-1]
    if values[1].startswith('Hurink') and values[2].startswith('s'):
        return 'Hurink_sdata_'+values[-1]
    if values[1].startswith('Hurink') and values[2].startswith('e'):
        return 'Hurink_edata_'+values[-1]
    if values[1].startswith('Hurink') and values[2].startswith('r'):
        return 'Hurink_rdata_'+values[-1]
    if values[1].startswith('Hurink') and values[2].startswith('v'):
        return 'Hurink_vdata_'+values[-1]
    if values[1].startswith('DP'):
        return 'DPpaulli'+values[-1]
    if values[1].startswith('Kacem'):
        return 'Kacem'+values[-1]
    if values[1].startswith('Fattahi'):
        return 'Fattahi'+values[-1]
    return name

In [21]:
import json
results_categories = os.listdir(path_to_results)
results_categories = ['dissimilarity', 'no_dissimilarity']
results = dict()
for category in results_categories:
    results[category] = dict()
    instances = os.listdir(path_to_results + category)
    for instance in instances:
        #with open(path_to_results + category + '\\' + instance, 'r') as f:
        #TODO: turn into valid json
        j_data = rewrite(category)
        for instance in j_data:
            ga_data = json.loads(j_data[instance])
            # TODO: gather best, average, ...
            if instance not in results[category]:
                results[category][remap(instance[:-17])] = ga_data


In [22]:
results['dissimilarity'].keys()

dict_keys(['Behnke42', 'Behnke46', 'Behnke60', 'BrandimarteMk12', 'BrandimarteMk14', 'BrandimarteMk7', 'Hurink_sdata_18', 'Hurink_sdata_1', 'Hurink_sdata_38', 'Hurink_sdata_40', 'Hurink_sdata_54', 'Hurink_sdata_61', 'Hurink_sdata_63', 'Hurink_edata_1', 'Hurink_edata_6', 'Hurink_rdata_28', 'Hurink_rdata_38', 'Hurink_rdata_50', 'Hurink_vdata_18', 'Hurink_vdata_5', 'DPpaulli15', 'DPpaulli18', 'DPpaulli1', 'DPpaulli9', 'ChambersBarnes10', 'Kacem3', 'Kacem4', 'Fattahi14', 'Fattahi20'])

In [23]:
benchmark_data['Hurink_edata_24']

{'n_operations': 150,
 'flexibility': 0.06133333333333333,
 'duration_variety': 0.07391304347826087,
 'n_machines': 10}

In [24]:
best_overall = dict()
average_overall = dict()
tied_overall = []
average_tied_overall = []
used = []
for instance in benchmark_data:
    if instance in results['dissimilarity'] or instance in results['no_dissimilarity']:
        best_overall[instance] = ('None', float('inf'))
        average_overall[instance] = ('None', float('inf'))
    for category in results:
        
        if instance in results[category]:
            if instance not in used: 
                used.append(instance)
            # TODO: decide what to do with instances with multiple best, also do same for average
            best = float('inf')
            average = 0
            for result in results[category][instance]['result']:
                if result['Result']['Fitness']['Makespan'] < best:
                    best = result['Result']['Fitness']['Makespan']
                average += result['Result']['Fitness']['Makespan']
            average /= len(results[category][instance]['result'])
            if best < best_overall[instance][1]:
                best_overall[instance] = (category, best)
            elif best == best_overall[instance][1]:
                tied_overall.append(instance)
                best_overall[instance] = ('None', float('inf'))
            if average < average_overall[instance][1]:
                average_overall[instance] = (category, average)
            elif average == average_overall[instance][1]:
                average_tied_overall.append(instance)
                average_overall[instance] = ('None', float('inf'))

In [25]:
len(used)

29

In [26]:
dissimilarity_win = [best_overall[x] for x in best_overall if best_overall[x][0] == 'dissimilarity']
no_dissimilarity_win = [best_overall[x] for x in best_overall if best_overall[x][0] == 'no_dissimilarity']
dissimilarity_average_win = [average_overall[x] for x in average_overall if average_overall[x][0] == 'dissimilarity']
no_dissimilarity_average_win = [average_overall[x] for x in average_overall if average_overall[x][0] == 'no_dissimilarity']

In [27]:
print(f'Dissimilarity better: {len(dissimilarity_win)}')
print(f'No Dissimilarity better: {len(no_dissimilarity_win)}')
print(f'Average Dissimilarity better: {len(dissimilarity_average_win)}')
print(f'Average No Dissimilarity better: {len(no_dissimilarity_average_win)}')

Dissimilarity better: 8
No Dissimilarity better: 17
Average Dissimilarity better: 4
Average No Dissimilarity better: 22


In [28]:
dissimilarity_average_win_instances = [x for x in average_overall if average_overall[x][0] == 'dissimilarity']

In [29]:
no_dissimilarity_average_win_instances = [x for x in average_overall if average_overall[x][0] == 'no_dissimilarity']

In [30]:
import statistics
compare = [dissimilarity_average_win_instances, no_dissimilarity_average_win_instances, average_tied_overall]
labels = ['dissimilarity', 'no_dissimilarity', 'tied']
for data, label in zip(compare, labels):
    ops = []
    f = []
    dv = []
    m = []
    for instance in data:
        ops.append(benchmark_data[instance]['n_operations'])
        f.append(benchmark_data[instance]['flexibility'])
        dv.append(benchmark_data[instance]['duration_variety'])
        m.append(benchmark_data[instance]['n_machines'])
    print(f'{label.upper()} - {len(ops)} - {(len(ops)/29)*100 :.2f}%')
    if len(ops) <= 1:            
        print(f'Operations: Mean: {ops[0]}, Median: {ops[0]}, Stdev: {0}')
        print(f'Machines: Mean: {m[0]}, Median: {m[0]}, Stdev: {0}')
        print(f'Flexibility: Mean: {f[0]}, Median: {f[0]}, Stdev: {0}')
        print(f'Duration Variety: Mean: {dv[0]}, Median: {dv[0]}, Stdev: {0}')
    else:
        print(f'Operations: Mean: {statistics.mean(ops)}, Median: {statistics.median(ops)}, Stdev: {statistics.stdev(ops)}, Min: {min(ops)}, Max: {max(ops)}')
        print(f'Machines: Mean: {statistics.mean(m)}, Median: {statistics.median(m)}, Stdev: {statistics.stdev(m)}, Min: {min(m)}, Max: {max(m)}')
        print(f'Flexibility: Mean: {statistics.mean(f)}, Median: {statistics.median(f)}, Stdev: {statistics.stdev(f)}, Min: {min(f)}, Max: {max(f)}')
        print(f'Duration Variety: Mean: {statistics.mean(dv)}, Median: {statistics.median(dv)}, Stdev: {statistics.stdev(dv)}, Min: {min(dv)}, Max: {max(dv)}')
    print()

DISSIMILARITY - 4 - 13.79%
Operations: Mean: 270, Median: 296.5, Stdev: 121.13078331566533, Min: 100, Max: 387
Machines: Mean: 8.25, Median: 9.0, Stdev: 2.362907813126304, Min: 5, Max: 10
Flexibility: Mean: 0.23885627833379738, Median: 0.2667093820644202, Stdev: 0.08968429742183073, Min: 0.10857777777777777, Max: 0.31342857142857145
Duration Variety: Mean: 0.015074420688435132, Median: 0.016251323040333256, Stdev: 0.006618764492887987, Min: 0.006509731720147291, Max: 0.02128530495292673

NO_DISSIMILARITY - 22 - 75.86%
Operations: Mean: 140.3181818181818, Median: 100.0, Stdev: 124.66464972787952, Min: 21, Max: 500
Machines: Mean: 15.318181818181818, Median: 10.0, Stdev: 18.39095627549782, Min: 4, Max: 60
Flexibility: Mean: 0.15947589988992972, Median: 0.13262465986394556, Stdev: 0.12133858769105411, Min: 0.0484, Max: 0.5384523809523809
Duration Variety: Mean: 0.16491126581794877, Median: 0.06142048838594401, Stdev: 0.223598630243733, Min: 6.217652662025811e-05, Max: 0.7348066298342542



In [31]:
dissimilarity_win_instances = [x for x in best_overall if best_overall[x][0] == 'dissimilarity']
no_dissimilarity_win_instances = [x for x in best_overall if best_overall[x][0] == 'no_dissimilarity']
compare = [dissimilarity_win_instances, no_dissimilarity_win_instances, tied_overall]
labels = ['dissimilarity', 'no_dissimilarity', 'tied']
for data, label in zip(compare, labels):
    ops = []
    f = []
    dv = []
    m = []
    for instance in data:
        ops.append(benchmark_data[instance]['n_operations'])
        f.append(benchmark_data[instance]['flexibility'])
        dv.append(benchmark_data[instance]['duration_variety'])
        m.append(benchmark_data[instance]['n_machines'])
    print(f'{label.upper()} - {len(ops)} - {(len(ops)/29)*100 :.2f}%')
    if len(ops) <= 1:            
        print(f'Operations: Mean: {ops[0]}, Median: {ops[0]}, Stdev: {0}')
        print(f'Machines: Mean: {m[0]}, Median: {m[0]}, Stdev: {0}')
        print(f'Flexibility: Mean: {f[0]}, Median: {f[0]}, Stdev: {0}')
        print(f'Duration Variety: Mean: {dv[0]}, Median: {dv[0]}, Stdev: {0}')
    else:
        print(f'Operations: Mean: {statistics.mean(ops)}, Median: {statistics.median(ops)}, Stdev: {statistics.stdev(ops)}, Min: {min(ops)}, Max: {max(ops)}')
        print(f'Machines: Mean: {statistics.mean(m)}, Median: {statistics.median(m)}, Stdev: {statistics.stdev(m)}, Min: {min(m)}, Max: {max(m)}')
        print(f'Flexibility: Mean: {statistics.mean(f)}, Median: {statistics.median(f)}, Stdev: {statistics.stdev(f)}, Min: {min(f)}, Max: {max(f)}')
        print(f'Duration Variety: Mean: {statistics.mean(dv)}, Median: {statistics.median(dv)}, Stdev: {statistics.stdev(dv)}, Min: {min(dv)}, Max: {max(dv)}')
    print()

DISSIMILARITY - 8 - 27.59%
Operations: Mean: 197, Median: 162.5, Stdev: 119.97380666509788, Min: 72, Max: 387
Machines: Mean: 9.625, Median: 10.0, Stdev: 2.774243783705493, Min: 5, Max: 15
Flexibility: Mean: 0.14427519088061708, Median: 0.08479696106362773, Stdev: 0.11713917956061892, Min: 0.03528619528619528, Max: 0.31342857142857145
Duration Variety: Mean: 0.12954582950231056, Median: 0.030489980720738174, Stdev: 0.2469133124455931, Min: 0.006509731720147291, Max: 0.7315175097276264

NO_DISSIMILARITY - 17 - 58.62%
Operations: Mean: 154.7058823529412, Median: 100, Stdev: 137.8113587054249, Min: 21, Max: 500
Machines: Mean: 17.176470588235293, Median: 10, Stdev: 20.664689975044528, Min: 4, Max: 60
Flexibility: Mean: 0.1845971526832828, Median: 0.1382857142857143, Stdev: 0.1271992994404529, Min: 0.05245596761842249, Max: 0.5384523809523809
Duration Variety: Mean: 0.15271301692533948, Median: 0.05004812319538017, Stdev: 0.2074716898426633, Min: 6.217652662025811e-05, Max: 0.7348066298342

In [32]:
print(max([benchmark_data[instance]['n_machines'] for instance in used]))
print(max([benchmark_data[instance]['n_operations'] for instance in used]))
print(max([benchmark_data[instance]['flexibility'] for instance in used]))
print(max([benchmark_data[instance]['duration_variety'] for instance in used]))

60
500
0.5384523809523809
0.7348066298342542


In [33]:
print(min([benchmark_data[instance]['n_machines'] for instance in used]))
print(min([benchmark_data[instance]['n_operations'] for instance in used]))
print(min([benchmark_data[instance]['flexibility'] for instance in used]))
print(min([benchmark_data[instance]['duration_variety'] for instance in used]))

4
21
0.03528619528619528
6.217652662025811e-05
